In [1]:
import os, sys, matplotlib, pylab
import numpy  as np
from astropy.io import fits
from matplotlib import pyplot as plt
import re
from astropy.table import Table, Column
from astropy.io import ascii
import scipy.stats as ss

def norm(a):
    p = (a-np.median(a))/np.median(a)
    return p
plt.switch_backend('Qt5Agg')

from astropy.stats import LombScargle

In [2]:
#start with path = /home/vatsal/UThesis/K2/K2_Refined/SC

def finalfilelist(path):
    os.chdir(path) # Change Directory
    nowpath = os.getcwd()
    filenamelist = filter(lambda x: x.startswith('ktwo'), os.listdir(nowpath))
    writelist = [ i +'\n' for i in filenamelist]
    with open('list.txt','w') as f:
        map( f.write , writelist)
    f.close()                                       #making the folder list

    Q10up = 'list.txt'
    Q10_file = file(Q10up).readlines()

    ktwoID = []
    ktwofilelist = []

    ktwo_detID = []
    ktwo_detfilelist = []

    for i in Q10_file:
        pathn = i.replace('\n','')
        os.chdir(path + "/%s"%pathn)

        foldername = i.strip().split(' ')
        folderlistfile = '%s_list.txt' %foldername[0]


        #1 Read names of all files ending with .fits
        nowpath = os.getcwd()
        filenamelist = filter(lambda x: x.endswith('.fits'), os.listdir(nowpath))

        #1.5 Write Raw data file into txt list (.fits)
        writelist = [ i +'\n' for i in filenamelist]
        with open('%s_Fits_list.txt' %foldername[0],'w') as f:
            map(f.write , writelist)

        #2 Read needed file data
        Kepler = '%s_Fits_list.txt' %foldername[0]
        LC_file= file(Kepler).readlines()



        for j in LC_file:
            if(j.startswith('ktwo')):
                a = re.findall('\d+', j)
                ktwoID.append(a[0])
                ktwofilelist.append(j)

            elif(j.startswith('EPIC')):
                a = re.findall('\d+', j)
                ktwo_detID.append(a[0])
                ktwo_detfilelist.append(j)

    com_ID =  [i for i in set(ktwoID) & set(ktwo_detID)]      #list of IDs common to both original and detrended list of files-
                                                              #essentially the ones that have to be analysed




    #Make a list of tuples containing names of common files from both folder

    ktwofilelist = zip([np.float(k) for k in ktwoID],ktwofilelist)
    ktwo_detfilelist = zip([np.float(l) for l in ktwo_detID],ktwo_detfilelist)


    ktwofilelist.sort(key=lambda x: x[0])
    ktwo_detfilelist.sort(key=lambda x: x[0])

    print len(ktwofilelist)
    print len(ktwo_detfilelist)

    comlist = zip([k[1] for k in ktwofilelist],[k[1] for k in ktwo_detfilelist])

    return comlist



In [3]:
#Start with nowpath = /home/vatsal/UThesis/K2/K2_Refined/SC
def plotprot(nowpath,comlist):
    numeric_const_pattern = r"""
    [-+]? # optional sign
    (?:
    (?: \d* \. \d+ ) # .1 .12 .123 etc 9.1 etc 98.1 etc
    |
    (?: \d+ \.? ) # 1. 12. 123. etc 1 12 123 etc
     )
    # followed by optional exponent part if desired
    (?: [Ee] [+-]? \d+ ) ?
    """
    rx = re.compile(numeric_const_pattern, re.VERBOSE)
    rot=[]
    for k,l in comlist:

        keplerid = re.findall('\d+',k)[0]
        k = k.replace('\n','')
        data1 = fits.open(nowpath+'/ktwo/'+k)
        dat=data1[1].data
        pdcsap_flux = dat['PDCSAP_FLUX']
        time_raw = dat['TIME']

        mnan1 = -np.isnan(pdcsap_flux)


        l = l.replace('\n','')
        data2 = fits.open(nowpath+'/ktwo-detrended/'+l)
        dat=data2[1].data
        time_det = dat['time']
        flux_det = dat['flux']
        trend_t = dat['trtime']
        trend_p = dat['trposi']
        x_cor = dat['x']
        y_cor = dat['y']

        mnan2 = -np.isnan(flux_det)

        output_file = l.replace('\n','') + '.rdb'
        fdata = Table([time_det,flux_det, trend_t, trend_p], names=['jdb', 'flux', 'trendt','trendp'])
        ascii.write(fdata,nowpath+'/ktwo-detrended/'+output_file , format='rdb')
        rot.append((keplerid,rx.findall(data2[1].header['KER_HPS1'])[2]))


        fluxerr = np.full((len(time_det[mnan2])),0.001)
        frequency, power = LombScargle(time_det[mnan2], 100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2]),\
                                       dy=fluxerr ,center_data=False).autopower()


        fig, ax = plt.subplots(4)
        #Plotting the x and y pixel centre values
        ax[0].plot(time_det[mnan2],x_cor[mnan2],'r.')
        ax[0].plot(time_det[mnan2],y_cor[mnan2],'k.')
        ax[0].set_ylabel('X(red) & Y(black)')
        ax[0].set_title(keplerid+' '+'Rotation period:'+rx.findall(data2[1].header['KER_HPS1'])[2])
        ax[0].axhline(y=0,xmin=0,xmax=1,color='k')

        #Plotting the Raw flux from original ktwo files
    #     ax[1].plot(time_raw[mnan1],100.*norm(pdcsap_flux[mnan1]),'k.-')
    #     ax[1].set_ylabel('Raw Relative flux[%]')


        #Plotting the supposedly detrended flux from the EPIC files
        ax[1].plot(time_det[mnan2],100.*norm(flux_det[mnan2]),'k.-')
        ax[1].set_ylabel('f[%]')

        #Plotting the light curves with corrections for systematics
        ax[2].plot(time_det[mnan2],100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2]),'k.-')
        ax[2].plot(time_det[mnan2],100.*norm(trend_t[mnan2]),'r')
        ax[2].set_ylabel('f-m(X)[%]')

        ax[3].plot(1/frequency, power)
        ax[3].set_xlabel(r'$Period(days)$')
        ax[3].set_ylabel(r'$ Power(Normalized)$')
        ax[3].set_xscale('log')


    #     #Plotting the light curves with corrections for both systematics and intrinsic stellar variations
    #     plt.plot(time_det[mnan2],100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2])-100.*norm(trend_t[mnan2]),'k.-')
    #     plt.ylabel('f-m(X)-m(t)[%]')
    #     plt.xlabel('Time')

        plt.show()


In [10]:
comlist_sc = finalfilelist('/home/vatsal/UThesis/K2/K2_Refined/SC')
comlist_misc_3000 = finalfilelist('/home/vatsal/UThesis/K2/data/kepler-3000')
comlist_misc_3002_3150 = finalfilelist('/home/vatsal/UThesis/K2/data/kepler-3002-3150')


30
30
285
255
140
92


In [11]:
# print comlist_sc
# print comlist_misc_2300_3000
# print comlist_misc_3002_3150

comsc1 = set(comlist_sc).intersection(comlist_misc_3000)
comsc2 = set(comlist_sc).intersection(comlist_misc_3002_3150)

print comsc1
print comsc2

set([('ktwo212820594-c06_llc.fits\n', 'EPIC_212820594_mast.fits\n'), ('ktwo206135809-c03_llc.fits\n', 'EPIC_206135809_mast.fits\n'), ('ktwo211046195-c04_llc.fits\n', 'EPIC_211046195_mast.fits\n'), ('ktwo206050032-c03_llc.fits\n', 'EPIC_206050032_mast.fits\n')])
set([('ktwo210758829-c04_llc.fits\n', 'EPIC_210758829_mast.fits\n')])


In [3]:
#Reading the Huber catalogue
dat = Table.read('/home/vatsal/UThesis/K2/K2_Refined/apjs522918t5_mrt.txt',format='ascii.cds')
dat.sort('Teff')
print dat.colnames

['EPIC', 'Teff', 'E_Teff', 'e_Teff', 'logg', 'E_logg', 'e_logg', '[Fe/H]', 'E_[Fe/H]', 'e_[Fe/H]', 'Radius', 'E_Radius', 'e_Radius', 'Mass', 'E_Mass', 'e_Mass', 'rho', 'E_rho', 'e_rho', 'Dist', 'E_Dist', 'e_Dist', 'E(B-V)', 'E_E(B-V)', 'e_E(B-V)', 'Flag']


In [14]:
#Extracting the M dwarf IDs for fetching the light curves from MAST
ind = [i for i,k in enumerate(dat['logg']) if(k>4 and 2300<=dat['Teff'][i]<=3800)]

writelist = [ str(i) +'\n' for i in dat['EPIC'][ind]]
print writelist
with open('huber_m_dwarfs_full.cat','w') as f:
    map(f.write , writelist)

with open('huber_m_dwarfs_1.cat','w') as f:
    map(f.write , writelist[0:10000])

with open('huber_m_dwarfs_2.cat','w') as f:
    map(f.write , writelist[10000:])


['211483380\n', '219635170\n', '218390978\n', '206386821\n', '211015471\n', '210753800\n', '211063591\n', '212820594\n', '219057545\n', '211749192\n', '212441506\n', '219173067\n', '219788678\n', '215758296\n', '212489420\n', '206381460\n', '210977069\n', '215054512\n', '214415560\n', '211021191\n', '220400139\n', '210715010\n', '211439891\n', '220171537\n', '214547615\n', '201721859\n', '219812088\n', '201884283\n', '201590753\n', '219293773\n', '215344447\n', '213849205\n', '215106360\n', '218471047\n', '219095470\n', '206135809\n', '212136544\n', '210467490\n', '215811764\n', '206131233\n', '212109980\n', '211046195\n', '215832018\n', '219079269\n', '219032628\n', '219745741\n', '215045531\n', '217919264\n', '219742197\n', '211435030\n', '219306155\n', '219432274\n', '201405113\n', '211973540\n', '212148445\n', '214885431\n', '219535682\n', '211191164\n', '212800574\n', '210954223\n', '215168099\n', '212428000\n', '219754034\n', '211778125\n', '219682954\n', '219777206\n', '21476823